### Notebook for initializing polymer systems using a DPD potential

In [1]:
import matplotlib
import numpy as np  
import gsd, gsd.hoomd 
import hoomd 
import time
import matplotlib_inline
import matplotlib.pyplot as plt
%matplotlib inline
matplotlib.style.use("ggplot")
matplotlib_inline.backend_inline.set_matplotlib_formats("svg")

In [49]:
def initialize_snapshot_rand_walk(num_pol, num_mon, density=0.85):
    '''
    Create a HOOMD snapshot of a cubic box with the number density given by input parameters.
    '''    
    N = num_pol * num_mon
    L = np.cbrt(N / density)  # Calculate box size based on density
    buffer = 4
    bond_length = 1.0
    print(L)
    positions = np.zeros((N, 3))
    for i in range(num_pol):
        start = i * num_mon
        positions[start] = np.random.uniform(low=(-L/2),high=(L/2),size=3)
        for j in range(num_mon - 1):
            delta = np.random.uniform(low=(-bond_length/2),high=(bond_length/2),size=3)
            delta /= np.linalg.norm(delta)*bond_length
            positions[start+j+1] = positions[start+j] + delta
    positions = pbc(positions,[L,L,L])
    bonds = []
    for i in range(num_pol):
        start = i * num_mon
        for j in range(num_mon - 1):
            bonds.append([start + j, start + j + 1])
    bonds = np.array(bonds)
    frame = gsd.hoomd.Frame()
    frame.particles.types = ['A']
    frame.particles.N = N
    frame.particles.position = positions
    frame.bonds.N = len(bonds)
    frame.bonds.group = bonds
    frame.bonds.types = ['b']
    frame.configuration.box = [L, L, L, 0, 0, 0]
    return frame

def pbc(d,box):
    for i in range(3):
        a = d[:,i]
        a[a < -box[i]/2] += box[i]
        a[a >  box[i]/2] -= box[i]
    return d

def check_bond_length_equilibration(gsdfile,bond_length,num_mon,num_pol): #todo
    fene_bond_l = bond_length
    traj = gsd.hoomd.open(gsdfile)
    last_frame = traj[-1]
    frame_ds = []
    for j in range(num_pol):
        idx = j*num_mon
        d1 = last_frame.particles.position[idx:idx+num_mon-1] - last_frame.particles.position[idx+1:idx+num_mon]
        bond_l = np.linalg.norm(pbc(d1,last_frame.configuration.box),axis=1)
        frame_ds.append(bond_l)
    max_frame_bond_l = np.max(np.array(frame_ds))
    min_frame_bond_l = np.min(np.array(frame_ds))
    print("max: ",max_frame_bond_l," min: ",min_frame_bond_l)
    if max_frame_bond_l <= fene_bond_l and min_frame_bond_l >= (fene_bond_l - 0.3):
        print("DPD Simulation Finished.")
        return True
    if max_frame_bond_l > fene_bond_l or min_frame_bond_l < (fene_bond_l - 0.3):
        return False

def run_one(A=1000,gamma=1000,k=1000,num_pol=100,num_mon=10,kT=1.0,r_cut = 1.15,bond_l=1.0,dt=0.001,density=0.8,particle_spacing = 1.1
):
    print(num_pol*num_mon)
    print(f"\nRunning with A={A}, gamma={gamma}, k={k}, "
          f"num_pol={num_pol}, num_mon={num_mon}")
    start_time = time.perf_counter()
    frame = initialize_snapshot_rand_walk(num_pol, num_mon, density=density)
    build_stop = time.perf_counter()
    print("Total build time: ", build_stop-start_time)
    harmonic = hoomd.md.bond.Harmonic()
    harmonic.params["b"] = dict(r0=bond_l, k=k)
    integrator = hoomd.md.Integrator(dt=dt)
    integrator.forces.append(harmonic)
    simulation = hoomd.Simulation(device=hoomd.device.auto_select(), seed=np.random.randint(65535))# TODO seed
    simulation.operations.integrator = integrator 
    simulation.create_state_from_snapshot(frame)
    const_vol = hoomd.md.methods.ConstantVolume(filter=hoomd.filter.All())
    integrator.methods.append(const_vol)
    nlist = hoomd.md.nlist.Cell(buffer=0.4)
    simulation.operations.nlist = nlist
    DPD = hoomd.md.pair.DPD(nlist, default_r_cut=r_cut, kT=kT)
    DPD.params[('A', 'A')] = dict(A=A, gamma=gamma)
    integrator.forces.append(DPD)

    gsd_file = f'run_len_{num_mon}_pol_{num_pol}_A{int(A)}_g{int(gamma)}_k{k}.gsd'
    gsd_out = hoomd.write.GSD(
        trigger=hoomd.trigger.Periodic(1000), 
        mode='wb',
        dynamic=['property','momentum'],
        filename=gsd_file,
        truncate=False)
    simulation.operations.writers.append(gsd_out)
    simulation.run(0)
    gsd_out.flush()
    simulation.run(1000)
    gsd_out.flush()
    while not check_bond_length_equilibration(gsd_file, particle_spacing, num_mon, num_pol): #TODO - work with snapshot instead of gsd?
        check_time = time.perf_counter()
        if (check_time-start_time) > 60:
            return num_pol*num_mon, 0
        simulation.run(1000)
        gsd_out.flush()
        print(sim.timestep)
    end_time = time.perf_counter()
    return num_pol*num_mon, end_time - start_time
    
N,s = run_one(A=1000,gamma=800,k=20000,num_pol=100,num_mon=50) #A=1000 seems fast, cranking up gamma to 800 speeds up, cranking up k more does too,
#no difference between k=10k and 20k
#OK, with these parameters, we're finishing in under 100 steps
print(f"Finished in time = {s:.2f}s")

5000

Running with A=1000, gamma=800, k=20000, num_pol=100, num_mon=50
18.420157493201934
Total build time:  0.04167358297854662
max:  1.0315987  min:  0.9744552
DPD Simulation Finished.
Finished in time = 0.94s


In [51]:
N,s = run_one(A=1000,gamma=800,k=20000,num_pol=100,num_mon=100)
print(f"Finished in time = {s:.2f}s")

10000

Running with A=1000, gamma=800, k=20000, num_pol=100, num_mon=100
23.207944168063896
Total build time:  0.0787271250737831
max:  1.0294348  min:  0.9690362
DPD Simulation Finished.
Finished in time = 1.86s


In [52]:
N,s = run_one(A=1000,gamma=800,k=20000,num_pol=100,num_mon=1000)
print(f"Finished in time = {s:.2f}s")

100000

Running with A=1000, gamma=800, k=20000, num_pol=100, num_mon=1000
50.0
Total build time:  0.5120189579902217
max:  1.0339326  min:  0.967937
DPD Simulation Finished.
Finished in time = 22.36s


In [53]:
N,s = run_one(A=1000,gamma=800,k=20000,num_pol=200,num_mon=1000)
print(f"Finished in time = {s:.2f}s")

200000

Running with A=1000, gamma=800, k=20000, num_pol=200, num_mon=1000
62.99605249474366
Total build time:  0.9821798749035224
max:  1.0381532  min:  0.963397
DPD Simulation Finished.
Finished in time = 45.62s
